In [6]:
import tensorflow as tf
import numpy as np

In [7]:
import os
cwd = os.getcwd()

#Project constants
NR_CLASSES = 6


#Control the graph being used by creating and using the "with" command. 
graph1 = tf.Graph()
with graph1.as_default():
    with tf.name_scope("placeholders"):
        x = tf.placeholder(tf.float32, (None, 200, 3))
        y = tf.placeholder(tf.float32, (None, 6))
        lr = tf.placeholder(tf.float32)
          
    with tf.name_scope("global_step"):
        # Creates a variable to hold the global_step. Global step is a variable that is updated every time training is started. 
        global_step = tf.Variable(0, trainable=False)
              
    with tf.name_scope("layers"):
        #Add any layers that constitute the network.
        x_flat = tf.contrib.layers.flatten(x)
        logits = tf.contrib.layers.fully_connected(x_flat,num_outputs = NR_CLASSES)#layer before softmax. 
        y_pred = tf.nn.softmax(logits, axis = 1)#last layer like a softmax. 
    
    with tf.name_scope("loss"):
        #Define the loss function that needs to be minimized. 
        entropy = tf.losses.softmax_cross_entropy(onehot_labels= y,logits= logits,reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS) #if redctuion is None shape is [batch_size], otherwise it is reduced to a scalar. i.e. sum over all the samples.
        l = tf.reduce_sum(entropy) #combined intropy of samples in batch. 
        
    with tf.name_scope("optimizer"):
        # define the an optimizer object.
        train_op = tf.train.AdamOptimizer(lr).minimize(l, global_step=global_step) #global step counts the ammount of training epochs that has happened. 
        
    with tf.name_scope("accuracy"):
        #Value usefull when comparing probability vector with one-hot label : [0; 0.30; 0.70] vs [0; 0; 1]
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1)) # check if largest value of label and prediction are the same. 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # calculate accuracy.
        
        
    with tf.name_scope("summary"):
        #Summary nodes( values that we want tot track ) 
        tf.summary.scalar('learning_rate', lr) 
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('loss', l)
        tf.summary.scalar('global_step', global_step)
        
        #Creates an object that needs to be called in sess.run() to run all summary nodes. 
        create_summary_op = tf.summary.merge_all()
        
        #Write log to file for tensorboard. Start tensorboard by: tensorboard --logdir=path/to/log-directory
        current_path =  os.path.abspath(os.getcwd())
        logdir = (current_path + '/log') ####### create variable for log file and graph)
        filewriter = tf.summary.FileWriter(logdir,graph1)

    with tf.name_scope("saver"):
        #Creates a saver object.
        saver = tf.train.Saver() 
        
    with tf.name_scope("initializer"):
        #Create an initializer object to be called when running.
        #If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables. Most high-level frameworks such as tf.contrib.slim, tf.estimator.Estimator and Keras automatically initialize variables for you before training a model.
        init_op = tf.global_variables_initializer()

Instructions for updating:
Use the retry module or similar alternatives.


In [8]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7744516244750398411, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 79167488
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 3299848771667172135
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 114819072
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 4041413039043931638
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [30]:
#Session
"""sess.run([fetch],{feed dict}): the fetch list determines the subgraphs to run in this session. The feeddict maps python datatypes to tf.tensors."""
def model_train(x_train, y_train, batch_size, nr_epochs, learning_rate, graph = tf.get_default_graph, x_test = None, y_test = None, weight_save_path = None):
    
    nr_samples = len(x_train)
    nr_batches_in_dataset = nr_samples // batch_size
     
    
    with tf.Session(graph= graph) as sess:
        for epoch_count in range(1,nr_epochs+1): 
            pos = 0

            print("nr_epoch:",epoch_count)

            for step in range(1,nr_batches_in_dataset+1):
                #create batch data
                x_batch= x_train[pos:pos+batch_size]
                y_batch= y_train[pos:pos+batch_size]
                #Training
                """During training for each minibatch is Minitbatch_Loss & Training accuracy are printed. If a test set is provided, the testing accuracy is calculated at the end of each epoch."""
                #sess.run(init_op) #initialize all variables if training is on. 
                sess.run(init_op)

                feed_dict = {x: x_batch, y: y_batch, lr: learning_rate} ### create method variables for these values. Be carefull not to use same names as in this example. 
                _,summary,loss,acc=sess.run([train_op, create_summary_op, l, accuracy],feed_dict) ### is  ceate_sumary_op not running l, accuracy already ? 

                print("step %d, minibatch_loss %f, minibatch_accuracy %f" %(step, loss, acc)) #print the values to the shell.

                # Calculate at the end of the epoch: on x_test, y_test the accuracy. 
                pos = pos + batch_size

            """End-of-epoch test set calculation."""
            if x_test is not None and y_test is not None: 
            # Checks if data is present. And calculates the Testing accuracy if present.
                ### Do we need to load values from model in some way????

                feed_dict = {x: x_test, y: y_test} ### create method variables for these values. Be carefull not to use same names as in this example. 
                epoch_test_acc=sess.run([accuracy],feed_dict)

                print("Test Set Accuracy:", epoch_test_acc)
            else: 
                print("No testing data was provided, can't calculate Testing Accuracy at the end of the epoch")
            print("--------------------------------------------------------------")
        if weight_save_path is not None:
            """There was a weight_save_path provided and we will write to it."""
            # Save the variables to disk.
            save_path = saver.save(sess, weight_save_path, global_step= global_step, write_meta_graph=False)
            print("Model saved in path: %s" % os.path.abspath(weight_save_path))

        else:
            """There was no save_path provided. We need to create one. """
            cwd = os.path.abspath(os.getcwd())
            weight_save_path = cwd+'/temp_model'

            save_path = saver.save(sess, weight_save_path, global_step= global_step, write_meta_graph=False)

            print("No path was provided, weights not saved. Model saved int path: %s"  % save_path)


        # Print the global step. 
        print('global_step: %s' % tf.train.global_step(sess, global_step))
        # Parting message.

        print("--------------------------------------------------------------")
        print("Training is finished")
        print("--------------------------------------------------------------")
        print("--------------------------------------------------------------")
    
        
def model_predict(x_eval,graph = tf.get_default_graph()):

    """ if weights are present, load them up."""
    sess = tf.Session(graph=graph) #deliberately start a session that is not closed because this allows to not have to save and reload the sessions weights every time we call this method. 
    #saver.restore(sess, weight_load_path)
    print("Model restored.")
    y_eval = sess.run([y_pred], feed_dict = {x: x_eval})
    return y_eval
    


### Test model

In [31]:
#toy dataset: 
k = np.transpose(np.array([range(0,200),range(0,200),range(0,200)]))
p = np.array(range(0,6))

x_train = np.array([k,k])
y_train =  np.array([p,p])
x_test = np.array([k,k])
y_test = np.array([p,p])

In [32]:
#toy input parameters
weight_save_path = os.getcwd()
batch_size = 2
nr_epochs = 30
learning_rate = 0.0001

In [33]:
model_train(x_train, y_train, batch_size = batch_size, nr_epochs=nr_epochs, learning_rate = learning_rate, x_test = x_test, y_test = y_test, weight_save_path = weight_save_path,graph = graph1)

nr_epoch: 1
step 1, minibatch_loss 66.884766, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 2
step 1, minibatch_loss 2118.448730, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 3
step 1, minibatch_loss 2549.537109, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 4
step 1, minibatch_loss 1928.696045, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 5
step 1, minibatch_loss 913.901550, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 6
step 1, minibatch_loss 1200.339355, minibatch_accuracy 0.000000
Test Set Accuracy: [0.0]
--------------------------------------------------------------
nr_epoch: 7
step 1,

In [34]:
model_predict(x_train)

Model restored.


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x7fb71be9e4e0>>
Traceback (most recent call last):
  File "/home/jgrietens/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 710, in __del__
    if self._session is not None:
AttributeError: 'Session' object has no attribute '_session'


RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().